In [1]:
from google.colab import files
uploaded = files.upload()


In [2]:
import pandas as pd

# Use the correct filename from the upload
df = pd.read_csv('/content/postings.csv')
df.head()


,job_title,company,job_location,job_link,first_seen,search_city,search_country,job level,job_type,job_summary,job_skills
0,Data Engineer 2,Cook Medical,"Bloomington, IN",https://www.linkedin.com/jobs/view/data-engine...,2023-12-17,Bloomington,United States,Mid senior,Onsite,"Overview\nThe Data Engineer develops, implemen...","Azure, SQL, NoSQL, SQL Server, Oracle, MongoDB..."
1,Staff Data Engineer,Recruiting from Scratch,"Bloomington, IN",https://www.linkedin.com/jobs/view/staff-data-...,2023-12-17,Bloomington,United States,Mid senior,Onsite,This is for a client of Recruiting from Scratc...,"Python, Snowflake, Airflow, Kubernetes, Docker..."
2,"Senior Data Engineer, Public Company",Recruiting from Scratch,"Bloomington, IN",https://www.linkedin.com/jobs/view/senior-data...,2023-12-17,Bloomington,United States,Mid senior,Onsite,This is for a client of Recruiting from Scratc...,"Python, SQL, Snowflake, Airflow, Kubernetes, D..."
3,"Senior Data Engineer, Public Company",Recruiting from Scratch,"Bloomington, IN",https://www.linkedin.com/jobs/view/senior-data...,2023-12-17,Bloomington,United States,Mid senior,Onsite,This is for a client of Recruiting from Scratc...,"TDD, Automation, Continuous delivery, Data eng..."
4,"Senior Systems Engineer, Azure Data Platform",Cook Medical,"Bloomington, IN",https://www.linkedin.com/jobs/view/senior-syst...,2023-12-17,Bloomington,United States,Mid senior,Hybrid,Overview\nWe are seeking a talented Azure Clou...,NaN


In [14]:
import pandas as pd
import re

# Load the dataset
df = pd.read_csv('postings.csv')

# Convert first_seen to datetime
df['first_seen'] = pd.to_datetime(df['first_seen'], errors='coerce', dayfirst=True)

# --- Skill Normalization Map ---
# Skill normalization map (grouping similar and inconsistent skill terms)
skill_map = {
    # SQL & ETL/ELT
    'structured query language': 'sql',
    'sql programming': 'sql',
    'etlpipelines': 'etl',
    'etl processes': 'etl',
    'elt processes': 'elt',

    # Data Warehousing
    'warehousing': 'datawarehouse',
    'data warehousing': 'datawarehouse',
    'data warehouse': 'datawarehouse',

    # BI Tools
    'power bi': 'powerbi',
    'Power BI': 'powerbi',
    'tableau software': 'tableau',

    # Data Engineering
    'data engineer': 'dataengineering',
    'de': 'dataengineering',

    # Pipelining
    'data pipelining': 'datapipeline',
    'data pipelines': 'datapipeline',

    # Spark
    'pyspark': 'spark',

    # Analytics
    'data analysis': 'dataanalytics',
    'dataanalysis': 'dataanalytics',
    'data analytics': 'dataanalytics',
    'data-analytics': 'dataanalytics',
    'data analyst': 'dataanalytics',

    # Visualization
    'data visualization': 'visualization',
    'visualization techniques': 'visualization',
    'datavisualization': 'visualization',

    # Dashboarding
    'dashboards': 'dashboard',
    'dashboarding': 'dashboard',

    # Modeling
    'datamodeling': 'datamodeling',
    'data modeling': 'datamodeling',

    # Maintenance & Cleaning
    'maintaining data': 'datamaintenance',
    'cleaning data': 'datacleaning',
    'data cleansing': 'datacleaning'
}


# Clean and normalize job_skills
def clean_and_normalize_skills(text):
    if pd.isna(text):
        return None
    text = re.sub(r'[^A-Za-z0-9, ]+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip().lower()
    skills = [skill.strip() for skill in text.split(',') if skill.strip()]
    normalized = [skill_map.get(skill, skill) for skill in skills]
    return ', '.join(sorted(set(normalized)))  # Remove duplicates

# Clean original (for reference)
def basic_clean(text):
    if pd.isna(text):
        return None
    text = re.sub(r'[^A-Za-z0-9, ]+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip().lower()
    skills = [skill.strip() for skill in text.split(',') if skill.strip()]
    return ', '.join(skills)

# Apply both raw and normalized versions
df['job_skills_raw'] = df['job_skills'].apply(basic_clean)
df['job_skills_cleaned'] = df['job_skills'].apply(clean_and_normalize_skills)

# Drop rows with no normalized skills
df = df[df['job_skills_cleaned'].notna() & (df['job_skills_cleaned'] != '')]

# Preview

df.info()

/tmp/ipython-input-14-49571404.py:8: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['first_seen'] = pd.to_datetime(df['first_seen'], errors='coerce', dayfirst=True)


<class 'pandas.core.frame.DataFrame'>
Index: 4960 entries, 0 to 6023
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   job_title           4960 non-null   object        
 1   company             4960 non-null   object        
 2   job_location        4960 non-null   object        
 3   job_link            4960 non-null   object        
 4   first_seen          4960 non-null   datetime64[ns]
 5   search_city         4960 non-null   object        
 6   search_country      4960 non-null   object        
 7   job level           4960 non-null   object        
 8   job_type            4960 non-null   object        
 9   job_summary         4960 non-null   object        
 10  job_skills          4960 non-null   object        
 11  job_skills_raw      4960 non-null   object        
 12  job_skills_cleaned  4960 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 542.5+

In [15]:
df.to_csv('postings_cleaned.csv', index=False)
